In [60]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib

In [272]:
def gen_sample_v0(length=-1):
    seq_length = length
    if seq_length == -1:
        seq_length = np.random.poisson(16)
    x = [np.random.choice([0,1,2],p=[0.4,0.4,0.2]) for i in range(seq_length)]
    y = []
    ps = []
    for i in range(len(x)):
        xscore = 0
        yscore = 0
        if x[i]==0:
            xscore += 5
        if x[i]==1:
            yscore += 5
        if i-3 >= 0 and x[i-3]==2:
            xscore += 10
            
        p = np.exp(xscore) / (np.exp(xscore) + np.exp(yscore))
        ps.append(p)
        y.append(np.random.choice([0, 1], p=[p,1-p]))
    return x,y

def gen_sample_v1(length=-1):
    seq_length = length
    if seq_length == -1:
        seq_length = np.random.poisson(16)
    x = [np.random.choice([0,1,2],p=[0.4,0.4,0.2]) for i in range(seq_length)]
    y = []
    ps = []
    xscore = 0
    yscore = 0
    for i in range(len(x)):        
        if x[i]==0:
            xscore += 3
        if x[i]==1:
            yscore += 3
        if i-3 >= 0 and x[i-3]==2:
            if x[i-1] == 1:
                yscore = 0
            if x[i-1] == 0:
                xscore = 0
        p = np.exp(xscore) / (np.exp(xscore) + np.exp(yscore))
        ps.append(p)
        y.append(np.random.choice([0, 1], p=[1-p, p]))
    return x,y,ps


def gen_sample_v2(length=-1, xp=-1, yp=-1):
    seq_length = length
    xpls, ypls = xp, yp
    if seq_length == -1:
        seq_length = np.random.poisson(16)
    if xpls == -1:
        xpls = np.exp(np.random.normal())
    if ypls == -1:
        ypls = np.exp(np.random.normal())
    x = [np.random.choice([0,1,2],p=[0.4,0.4,0.2]) for i in range(seq_length)]
    y = []
    ps = []
    xscore = 0
    yscore = 0
    for i in range(len(x)):        
        if x[i]==0:
            xscore += xpls
        if x[i]==1:
            yscore += ypls
        if i-3 >= 0 and x[i-3]==2:
            if x[i-1] == 1:
                yscore = 0
            if x[i-1] == 0:
                xscore = 0
        p = np.exp(xscore) / (np.exp(xscore) + np.exp(yscore))
        ps.append(p)
        y.append(np.random.choice([0, 1], p=[1-p, p]))
    return x,y,ps

In [217]:
train_x, train_y, _ = zip(*[gen_sample_v1(32) for i in range(10)])

In [207]:
src_length = 16
num_units = 10

tf.reset_default_graph()

tf_src_x = tf.placeholder(shape=(None, src_length), dtype=tf.int32)
tf_src_y = tf.placeholder(shape=(None, src_length), dtype=tf.int32)
tf_x = tf.one_hot(tf_src_x, 3)
tf_y = tf.one_hot(tf_src_y, 2)

tf_cell = tf.nn.rnn_cell.GRUCell(num_units=num_units)
#tf_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=num_units, activation=tf.nn.relu) for _ in range(16)])

tf_out, _ = tf.nn.dynamic_rnn(tf_cell, inputs=tf_x, dtype=tf.float32)
tf_res = tf.layers.dense(tf_out, 2, use_bias=True)

tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=tf_res, labels=tf_y))
tf_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y,axis=2), tf.argmax(tf_res,axis=2)), dtype=tf.float32))
tf_prob = tf.nn.softmax(tf_res)
tf_train = tf.train.AdamOptimizer(1e-2).minimize(tf_loss)

tf.summary.FileWriter('D:/Jupyter/Logs/09A_SeqRNN_1', tf.get_default_graph())

In [208]:
num_steps = 200
num_epochs = 20
seq_length = 16
batch_size = 200
valid_size = 1000


valid_x, valid_y, valid_p = zip(*[gen_sample_v0(seq_length) for i in range(valid_size)])
valid_batch = {tf_src_x: valid_x, tf_src_y: valid_y}

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(num_epochs):
        train_x, train_y, _ = zip(*[gen_sample_v0(seq_length) for i in range(batch_size)])
        train_batch = {tf_src_x: train_x, tf_src_y: train_y}

        start_loss = tf_loss.eval(feed_dict=train_batch)
        for j in range(num_steps):
            tf_train.run(feed_dict=train_batch)
        
        end_loss = tf_loss.eval(feed_dict=train_batch)
        valid_loss = tf_loss.eval(feed_dict=valid_batch)
        
        print('Step {0}: Train {1:1.3} -> {2:1.3}, Validation: {3}'.format(i, start_loss, end_loss, valid_loss))
    final_p, final_loss, final_accuracy = sess.run([tf_prob, tf_loss, tf_accuracy], feed_dict=valid_batch)
print('Final Cross-Entropy Loss: {0:1.4}\nFinal Accuracy: {1:1.4}'.format(final_loss, final_accuracy))

Step 0: Train 0.686 -> 0.112, Validation: 0.1711123138666153
Step 1: Train 0.147 -> 0.0876, Validation: 0.2341870218515396
Step 2: Train 0.257 -> 0.102, Validation: 0.2326425164937973
Step 3: Train 0.236 -> 0.104, Validation: 0.21845898032188416
Step 4: Train 0.243 -> 0.121, Validation: 0.19225411117076874
Step 5: Train 0.184 -> 0.104, Validation: 0.19973984360694885
Step 6: Train 0.204 -> 0.11, Validation: 0.20690733194351196
Step 7: Train 0.207 -> 0.109, Validation: 0.21080730855464935
Step 8: Train 0.223 -> 0.119, Validation: 0.2004842907190323
Step 9: Train 0.19 -> 0.101, Validation: 0.20857974886894226
Step 10: Train 0.189 -> 0.103, Validation: 0.20632463693618774
Step 11: Train 0.201 -> 0.114, Validation: 0.19410723447799683
Step 12: Train 0.164 -> 0.112, Validation: 0.17720364034175873
Step 13: Train 0.181 -> 0.117, Validation: 0.1877427101135254
Step 14: Train 0.176 -> 0.115, Validation: 0.186659038066864
Step 15: Train 0.197 -> 0.128, Validation: 0.18094901740550995
Step 16: T

## Results for V-0
### BasicRNNCell
1) 1 unit $\rightarrow$ Loss = $0.3230$, Accuracy = $0.8501$

2) 2 units $\rightarrow$ Loss = $0.2982$, Accuracy = $0.8465$

3) 3 units $\rightarrow$ Loss = $0.1743$, Accuracy = $0.9051$

4) 4 units $\rightarrow$ Loss = $0.1536$, Accuracy = $0.9103$ (Almost instant convergence)

5) 5 units $\rightarrow$ Loss = $0.1490$, Accuracy = $0.9089$ (Almost instant convergence)

6) 10 units $\rightarrow$ Loss = $0.1681$, Accuracy = $0.9084$ (Almost instant convergence & overtraining)

7) 30 units $\rightarrow$ Loss = $0.2426$, Accuracy = $0.9071$ (No convergence, overtraining on train sample)

### BasicLSTMCell
1) 1 unit $\rightarrow$ Loss = $0.2911$, Accuracy = $0.8503$

2) 2 units $\rightarrow$ Loss = $0.2168$, Accuracy = $0.8944$ (Slow convergence)

3) 3 units $\rightarrow$ Loss = $0.1515$, Accuracy = $0.9087$

4) 4 units $\rightarrow$ Loss = $0.1680$, Accuracy = $0.9134$ 

5) 5 units $\rightarrow$ Loss = $0.1732$, Accuracy = $0.9100$ 

6) 10 units $\rightarrow$ Loss = $0.1855$, Accuracy = $0.9113$ (Overtraining)

### GRUCell
1) 1 unit $\rightarrow$ Loss = $0.3166$, Accuracy = $0.8268$

2) 2 units $\rightarrow$ Loss = $0.2756$, Accuracy = $0.8586$

3) 3 units $\rightarrow$ Loss = $0.1854$, Accuracy = $0.9062$

4) 10 units $\rightarrow$ Loss = $0.1778$, Accuracy = $0.9047$ (Overtraining)

In [267]:
src_length = 16
num_units = 10

tf.reset_default_graph()

tf_src_x = tf.placeholder(shape=(None, src_length), dtype=tf.int32)
tf_src_y = tf.placeholder(shape=(None, src_length), dtype=tf.int32)
tf_x = tf.one_hot(tf_src_x, 3)
tf_y = tf.one_hot(tf_src_y, 2)

tf_cell = tf.nn.rnn_cell.LSTMCell(num_units=num_units)
#tf_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=num_units, activation=tf.nn.relu) for _ in range(16)])

tf_out, _ = tf.nn.dynamic_rnn(tf_cell, inputs=tf_x, dtype=tf.float32)
tf_res = tf.layers.dense(tf_out, 2, use_bias=True)

tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=tf_res, labels=tf_y))
tf_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y,axis=2), tf.argmax(tf_res,axis=2)), dtype=tf.float32))
tf_prob = tf.nn.softmax(tf_res)
tf_train = tf.train.AdamOptimizer(1e-2).minimize(tf_loss)

tf.summary.FileWriter('D:/Jupyter/Logs/09A_SeqRNN_2', tf.get_default_graph())

In [268]:
num_steps = 200
num_epochs = 5
seq_length = 16
batch_size = 200
valid_size = 1000


valid_x, valid_y, valid_p = zip(*[gen_sample_v1(seq_length) for i in range(valid_size)])
valid_batch = {tf_src_x: valid_x, tf_src_y: valid_y}

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(num_epochs):
        train_x, train_y, _ = zip(*[gen_sample_v1(seq_length) for i in range(batch_size)])
        train_batch = {tf_src_x: train_x, tf_src_y: train_y}

        start_loss = tf_loss.eval(feed_dict=train_batch)
        for j in range(num_steps):
            tf_train.run(feed_dict=train_batch)
        
        end_loss = tf_loss.eval(feed_dict=train_batch)
        valid_loss = tf_loss.eval(feed_dict=valid_batch)
        
        print('Step {0}: Train {1:1.3} -> {2:1.3}, Validation: {3}'.format(i, start_loss, end_loss, valid_loss))
    final_p, final_loss, final_accuracy = sess.run([tf_prob, tf_loss, tf_accuracy], feed_dict=valid_batch)
print('Final Cross-Entropy Loss: {0:1.4}\nFinal Accuracy: {1:1.4}'.format(final_loss, final_accuracy))
max_accuracy = np.mean(np.argmax(np.array([1-np.array(valid_p), np.array(valid_p)]), axis=0)==np.array(valid_y))
print('Maximum possible accuracy: {0:1.4}'.format(max_accuracy))

Step 0: Train 0.707 -> 0.213, Validation: 0.2992643117904663
Step 1: Train 0.292 -> 0.169, Validation: 0.27766871452331543
Step 2: Train 0.269 -> 0.167, Validation: 0.2666035294532776
Step 3: Train 0.259 -> 0.173, Validation: 0.2639874815940857
Step 4: Train 0.269 -> 0.167, Validation: 0.2515926957130432
Final Cross-Entropy Loss: 0.2516
Final Accuracy: 0.8885
Maximum possible accuracy: 0.9


In [276]:
src_length = 16
num_units = 16

tf.reset_default_graph()

tf_src_x = tf.placeholder(shape=(None, src_length), dtype=tf.int32)
tf_src_y = tf.placeholder(shape=(None, src_length), dtype=tf.int32)
tf_x = tf.one_hot(tf_src_x, 3)
tf_y = tf.one_hot(tf_src_y, 2)

tf_cell = tf.nn.rnn_cell.LSTMCell(num_units=num_units)
#tf_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=num_units, activation=tf.nn.relu) for _ in range(16)])

tf_out, _ = tf.nn.dynamic_rnn(tf_cell, inputs=tf_x, dtype=tf.float32)
tf_res = tf.layers.dense(tf_out, 2, use_bias=True)

tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=tf_res, labels=tf_y))
tf_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y,axis=2), tf.argmax(tf_res,axis=2)), dtype=tf.float32))
tf_prob = tf.nn.softmax(tf_res)
tf_train = tf.train.AdamOptimizer(1e-2).minimize(tf_loss)

tf.summary.FileWriter('D:/Jupyter/Logs/09A_SeqRNN_2', tf.get_default_graph())

In [277]:
num_steps = 200
num_epochs = 5
seq_length = 16
batch_size = 200
valid_size = 1000


valid_x, valid_y, valid_p = zip(*[gen_sample_v2(seq_length) for i in range(valid_size)])
valid_batch = {tf_src_x: valid_x, tf_src_y: valid_y}

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(num_epochs):
        train_x, train_y, _ = zip(*[gen_sample_v2(seq_length) for i in range(batch_size)])
        train_batch = {tf_src_x: train_x, tf_src_y: train_y}

        start_loss = tf_loss.eval(feed_dict=train_batch)
        for j in range(num_steps):
            tf_train.run(feed_dict=train_batch)
        
        end_loss = tf_loss.eval(feed_dict=train_batch)
        valid_loss = tf_loss.eval(feed_dict=valid_batch)
        
        print('Step {0}: Train {1:1.3} -> {2:1.3}, Validation: {3}'.format(i, start_loss, end_loss, valid_loss))
    final_p, final_loss, final_accuracy = sess.run([tf_prob, tf_loss, tf_accuracy], feed_dict=valid_batch)
print('Final Cross-Entropy Loss: {0:1.4}\nFinal Accuracy: {1:1.4}'.format(final_loss, final_accuracy))
max_accuracy = np.mean(np.argmax(np.array([1-np.array(valid_p), np.array(valid_p)]), axis=0)==np.array(valid_y))
print('Maximum possible accuracy: {0:1.4}'.format(max_accuracy))

Step 0: Train 0.689 -> 0.441, Validation: 0.752077579498291
Step 1: Train 0.738 -> 0.411, Validation: 0.7948712110519409
Step 2: Train 0.687 -> 0.39, Validation: 0.8525694608688354
Step 3: Train 0.808 -> 0.394, Validation: 0.7713333964347839
Step 4: Train 0.77 -> 0.377, Validation: 0.8468413352966309
Final Cross-Entropy Loss: 0.8468
Final Accuracy: 0.6666
Maximum possible accuracy: 0.841
